# 트리의 앙상블

## 랜덤포레스트

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv("https://bit.ly/wine_csv_data")

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

In [8]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 훈련 세트에 다소 과대적합이 됐다.

0.9973541965122431 0.8905151032797809


In [28]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

# [알코올 도수, 당도, pH]였는데 1절의 결정트리에서보다 당도의 중요도가 감소하고 알콜도수와 pH특성의 중요도가
# 조금 상승했다. 이유는 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문이다.
# 그 결과 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻었다.
# 이는 과대적합을 줄이고 일반화 성능을 높이는 데 도움이 된다.

[0.23167441 0.50039841 0.26792718]


In [12]:
rf = RandomForestClassifier(oob_score=True, n_jobs = -1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

# OOB점수를 사용하면 교차검증을 대신할 수 있어서 결과적으로 훈련세트에 더 많은 샘플을
# 사용할 수 있다.

0.8934000384837406


## 엑스트라 트리

In [14]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = - 1, random_state = 42)
scores = cross_validate(et, train_input, train_target, return_train_score = True, n_jobs = -1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 랜덤포레스트와 비슷한 결과를 얻었다. 

0.9974503966084433 0.8887848893166506


In [15]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅

In [16]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 엑스트라 트리와 랜덤포레스트의 앙상블과 달리 레이디언트 앙상블은 
# 과대적합에 매우 강하다.

0.8881086892152563 0.8720430147331015


In [18]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate = 0.2, random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 결정 트리 개수를 500개로 5배나 늘렸지만 과대적합을잘 억제하고 있다. 
# 학습률 learning_rate의 기본값은 0.1이다. 그레이디언트 부스팅도 특성 중요도를 제공한다.
# 결과에서 볼 수 있듯이 그레이디언트 부스팅이 랜덤 포레스트보다 일부 특성(당도)에 더 집중한다.

0.9464595437171814 0.8780082549788999


In [20]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


## 히스토그램 기반 그레이디언트 부스팅

In [24]:
# 사이킷런 1.0 버전 아래에서는 다음 라인의 주석을 해제하고 실행하세요.
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [26]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats = 10, random_state = 42, n_jobs = -1)
print(result.importances_mean)

# 평균을 출력해보면 랜덤 포레스트와 비슷한 비율임을 알 수 있다.

[0.08876275 0.23438522 0.08027708]


In [30]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

# 이것은 테스트 세트에서의 특성 중요도를 계산해본것,
# 그레이디언트 부스팅과 비슷하게 당도에 더 집중하고 있다.

[0.05969231 0.20238462 0.049     ]


In [31]:
hgb.score(test_input, test_target)

# 테스트 세트에서의 성능을 최종적으로 확인한것
# 약 87%의 정확도를 얻었다.

0.8723076923076923

### XGBoost

In [32]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist', random_state = 42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 와인 데이터의 교차검증 점수를 확인해보겠다.

0.8824322471423747 0.8726214185237284


### LightGBM

In [33]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
